In [3]:
import iatikit

In [4]:
iatikit.__version__

'2.1.0'

In [1]:
import sys

!{sys.executable} -m pip install --upgrade iatikit

  Found existing installation: iatikit 2.0.5
    Uninstalling iatikit-2.0.5:
      Successfully uninstalled iatikit-2.0.5


In [3]:
iatikit.download.data()

In [1]:
from datetime import datetime
import iatikit

registry = iatikit.data()

publishers = sorted(
    [(min([d.metadata.get('metadata_created')
           for d in p.datasets]
          ), p.metadata.get('title'))
     for p in registry.publishers])

for idx, tup in enumerate(publishers):
    print('{order}: {name} ({date})'.format(
        order=(idx + 1),
        name=tup[1],
        date=datetime.strptime(tup[0], '%Y-%m-%dT%H:%M:%S.%f').date()
    ))

1: UK - Department for International Development (DFID) (2011-01-29)
2: The William and Flora Hewlett Foundation (2011-03-31)
3: The World Bank (2011-05-14)
4: Development Initiatives Poverty Research (2011-07-13)
5: Australia - Department of  Foreign Affairs and Trade (2011-09-02)
6: Netherlands - Ministry of Foreign Affairs (2011-09-19)
7: United Nations Office for Project Services (UNOPS) (2011-10-03)
8: Sweden, through Swedish International Development Cooperation Agency (Sida) (2011-11-11)
9: Oxfam GB (2011-11-25)
10: Finland - Ministry of Foreign Affairs (2011-11-25)
11: The Global Fund to Fight AIDS, Tuberculosis and Malaria (2011-11-25)
12: Asian Development Bank (2011-11-28)
13: Publish What You Fund (2011-11-29)
14: Resource Extraction Monitoring (2012-02-21)
15: ADRA-UK (2012-02-24)
16: Catholic Agency For Overseas Development (CAFOD) (2012-03-21)
17: GAVI Alliance (2012-03-30)
18: GlobalGiving.org (2012-05-09)
19: LEAD International (2012-05-24)
20: MapAction (2012-05-28)
2

507: Trianglen (2017-04-28)
508: HealthNet TPO (2017-05-02)
509: Food and Agriculture Organization of the United Nations (FAO) (2017-05-03)
510: International Union for Conservation of Nature and Natural Resources (2017-05-15)
511: Meningitis Research Foundation (2017-05-18)
512: International Network for Bamboo and Rattan (INBAR) (2017-05-19)
513: Arab foundation for freedoms and equality (2017-05-24)
514: Near East Foundation (NEF USA) (2017-06-07)
515: World Economic Forum Geneva (2017-06-08)
516: World Health Organization (2017-06-08)
517: Giro555 (Stichting Samenwerkende Hulporganisaties) (2017-06-12)
518: Support Trust for Africa Development (STAD) (2017-06-13)
519: Netherlands Enterprise Agency (2017-06-15)
520: EMMS International (2017-06-23)
521: MVO Nederland (2017-06-27)
522: PUM Netherlands (2017-06-27)
523: People in Need (2017-06-28)
524: Disability Rights Advocacy Fund (2017-06-29)
525: AICS - Agenzia Italiana per la Cooperazione allo Sviluppo / Italian Agency for Cooper

In [37]:
import iatikit

# get the AsDB publisher
asdb = registry.publishers.find(name='asdb')

activities = asdb.activities

total_c = 0
total_d_e = 0

for activity in activities:  
    values = activity.etree.xpath('transaction[transaction-type/@code="C"]/value/text()')
    total_c += sum([float(v) for v in values])

for activity in activities:  
    values = activity.etree.xpath('transaction[transaction-type/@code="D" or transaction-type/@code="E"]/value/text()')
    total_d_e += sum([float(v) for v in values])

print(total_c)        
print(total_d_e)

156554932323.78998
92602825355.02998


In [14]:
import iatikit

registry = iatikit.data()

dfid = registry.publishers.find(name='dfid')
sector_category = iatikit.Sector(311, 2)  # Agriculture

ag_acts = dfid.activities.where(
    actual_start__lte='2017-12-31',  # started before 2018
    actual_end__gte='2017-01-01',  # ended after 2016
    sector__in=sector_category,
)
print('DFID had {:,} agricultural activities running during 2017.'.format(
    len(ag_acts)))

DFID had 201 agricultural activities running during 2017.


In [4]:
transaction_types = ['D', 'E', '3', '4']
' or '.join(['transaction-type/@code="{transaction_type}"'.format(
        transaction_type=transaction_type,
    ) for transaction_type in transaction_types])

'transaction-type/@code="D" or transaction-type/@code="E" or transaction-type/@code="3" or transaction-type/@code="4"'

In [15]:
import iatikit
from collections import defaultdict

registry = iatikit.data()

def total_transactions(transaction_name, org_name, year):
    if transaction_name == 'commitments':
        transaction_types = ['C', '2']
    elif transaction_name == 'spend':
        transaction_types = ['D', 'E', '3', '4']
    else:
        raise Exception(f'Unknown transaction name: {transaction_name}')
    org = registry.publishers.find(name=org_name)
    activities = org.activities

    total = defaultdict(float)

    type_str = ' or '.join([f'transaction-type/@code="{transaction_type}"'
                            for transaction_type in transaction_types])
    
    path = f'transaction[{type_str}]' + \
           f'[number(translate(transaction-date/@iso-date, "-", "")) >= {year}0101 ' + \
           f'and number(translate(transaction-date/@iso-date, "-", "")) < {year+1}0101]' + \
           f'/value'
    for activity in activities:
        values = activity.etree.xpath(path)
        default_currency = activity.etree.get('default-currency')
        for value in values:
            currency = value.get('currency') or default_currency
            total[currency] += float(value.text)

    print(f'Sum of {transaction_name} for {org_name} in {year}:')
    if total == {}:
        print('  No transactions of that type were found')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': None,
            'value': None,
        }]
    else:
        for currency, value in total.items():
            print(f'  {currency} {value:,.0f}')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': currency,
            'value': value,
        } for currency, value in total.items()]
    
    return out

In [16]:
import csv
fieldnames = ['org_name', 'year', 'transaction_type', 'currency', 'value']

org_names = [
    'asdb',
    'undp',
    'dfid',
    'afdb',
    'millenniumchallenge',
    'worldbank',
    'iadb',
    'gac-amc',
    'unicef',
    'ec-echo',
    'ec-near',
    'gavi',
    'theglobalfund',
    'sida',
    'minbuza_nl',
    'usaid',
    'ec-devco',
    'be-dgd',
    'unitedstates',
    'bmz',
    'ausgov',
    'ebrd',
    'eib',
    'finland_mfa',
    'ifcwbg',
    'sdc_ch',
    'danida',
    'bmgf',
    'france',
    'afd',
    'aics',
    'norad',
    'irishaid',
    'jica',
    'odakorea',
    'maec',
    'fco',
    'unocha',
    'mfat',
    'mofa-jpn',

]
with open('output.csv', 'w') as handler:
    writer = csv.DictWriter(handler, fieldnames=fieldnames)
    writer.writeheader()
    for org_name in org_names:
        for transaction_name in ['commitments', 'spend']:
            for year in [2018]:
                    output = total_transactions(transaction_name, org_name, year)
                    for o in output:
                        writer.writerow(o)

Sum of commitments for asdb in 2018:
  en 18,441,524,000
Sum of spend for asdb in 2018:
  en 11,822,441,908
Sum of commitments for undp in 2018:
  No transactions of that type were found
Sum of spend for undp in 2018:
  USD 4,364,096,701
Sum of commitments for dfid in 2018:
  GBP 4,608,209,615
Sum of spend for dfid in 2018:
  GBP 9,107,089,268
Sum of commitments for afdb in 2018:
  XDR 8,032,760,223
Sum of spend for afdb in 2018:
  XDR 3,436,171,472
Sum of commitments for millenniumchallenge in 2018:
  USD 481,533,342
Sum of spend for millenniumchallenge in 2018:
  USD 698,978,600
Sum of commitments for worldbank in 2018:
  USD 40,859,346,243
Sum of spend for worldbank in 2018:
  USD 33,080,579,691
Sum of commitments for iadb in 2018:
  USD 5,914,951,000
Sum of spend for iadb in 2018:
  USD 9,809,366,094
Sum of commitments for gac-amc in 2018:
  CAD 1,568,554,451
Sum of spend for gac-amc in 2018:
  CAD 2,723,379,298
Sum of commitments for unicef in 2018:
  USD 5,254,239,924
Sum of spen

Sum of spend for ec-echo in 2018:
  EUR 1,651,637,238


Sum of commitments for ec-near in 2018:
  EUR 3,066,222,154
Sum of spend for ec-near in 2018:
  EUR 2,640,078,542
Sum of commitments for gavi in 2018:
  USD 1,532,854,677
Sum of spend for gavi in 2018:
  USD 1,343,011,152


Sum of commitments for theglobalfund in 2018:
  USD 2,387,718,257


Sum of spend for theglobalfund in 2018:
  USD 1,519,746,652


...

Sum of spend for afd in 2018:
  EUR 1,607,320,288


Sum of commitments for aics in 2018:
  EUR 1,089,594,572
Sum of spend for aics in 2018:
  EUR 488,409,950
Sum of commitments for norad in 2018:
  NOK 19,655,173,497
Sum of spend for norad in 2018:
  NOK 24,511,058,224
Sum of commitments for irishaid in 2018:
  No transactions of that type were found


Sum of spend for irishaid in 2018:
  No transactions of that type were found


Sum of commitments for jica in 2018:
  No transactions of that type were found


Sum of spend for jica in 2018:
  No transactions of that type were found
Sum of commitments for odakorea in 2018:
  KRW 3,466,046,437,181
Sum of spend for odakorea in 2018:
  KRW 2,423,926,636,149
Sum of commitments for maec in 2018:
  No transactions of that type were found
Sum of spend for maec in 2018:
  No transactions of that type were found
Sum of commitments for fco in 2018:
  No transactions of that type were found
Sum of spend for fco in 2018:
  GBP 969,322,701
Sum of commitments for unocha in 2018:
  No transactions of that type were found
Sum of spend for unocha in 2018:
  No transactions of that type were found
Sum of commitments for mfat in 2018:
  No transactions of that type were found
Sum of spend for mfat in 2018:
  No transactions of that type were found
Sum of commitments for mofa-jpn in 2018:
  No transactions of that type were found
Sum of spend for mofa-jpn in 2018:
  No transactions of that type were found


In [8]:
import iatikit
from collections import defaultdict

registry = iatikit.data()

def total_transactions(transaction_name, org_name, year):
    if transaction_name == 'commitments':
        transaction_types = ['C', '2']
    elif transaction_name == 'spend':
        transaction_types = ['D', 'E', '3', '4']
    else:
        raise Exception(f'Unknown transaction name: {transaction_name}')
    org = registry.publishers.find(name=org_name)
    activities = org.activities

    total = defaultdict(float)

    type_str = ' or '.join([f'transaction-type/@code="{transaction_type}"'
                            for transaction_type in transaction_types])
    
    path = f'transaction[{type_str}]' + \
           f'[number(translate(transaction-date/@iso-date, "-", "")) >= {year}0406 ' + \
           f'and number(translate(transaction-date/@iso-date, "-", "")) < {year+1}0406]' + \
           f'/value'
    for activity in activities:
        values = activity.etree.xpath(path)
        default_currency = activity.etree.get('default-currency')
        for value in values:
            currency = value.get('currency') or default_currency
            total[currency] += float(value.text)

    print(f'Sum of {transaction_name} for {org_name} in {year}:')
    if total == {}:
        print('  No transactions of that type were found')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': None,
            'value': None,
        }]
    else:
        for currency, value in total.items():
            print(f'  {currency} {value:,.0f}')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': currency,
            'value': value,
        } for currency, value in total.items()]
    
    return out

In [9]:
import csv
fieldnames = ['org_name', 'year', 'transaction_type', 'currency', 'value']

org_names = [
    'beis',
    'fco',
    'defra_transparency',
    'dwp',
    'dhsc',
    'hooda',
    'ukmod',
   
]
with open('output.csv', 'w') as handler:
    writer = csv.DictWriter(handler, fieldnames=fieldnames)
    writer.writeheader()
    for org_name in org_names:
        for transaction_name in ['commitments', 'spend']:
            for year in [2018, 2017]:
                    output = total_transactions(transaction_name, org_name, year)
                    for o in output:
                        writer.writerow(o)

Sum of commitments for beis in 2018:
  No transactions of that type were found
Sum of commitments for beis in 2017:
  No transactions of that type were found
Sum of spend for beis in 2018:
  No transactions of that type were found
Sum of spend for beis in 2017:
  No transactions of that type were found
Sum of commitments for fco in 2018:
  No transactions of that type were found
Sum of commitments for fco in 2017:
  No transactions of that type were found
Sum of spend for fco in 2018:
  GBP 714,787,204
Sum of spend for fco in 2017:
  GBP 886,569,095
Sum of commitments for defra_transparency in 2018:
  No transactions of that type were found
Sum of commitments for defra_transparency in 2017:
  No transactions of that type were found
Sum of spend for defra_transparency in 2018:
  No transactions of that type were found
Sum of spend for defra_transparency in 2017:
  GBP 11,262,955
Sum of commitments for dwp in 2018:
  No transactions of that type were found
Sum of commitments for dwp in 2

In [32]:
import iatikit
from collections import defaultdict

registry = iatikit.data()

import csv
fieldnames = ['name', 'period-start', 'period-end', 'value', 'currency']

pub_names = [
    'asdb',
    'undp',
    'dfid',
    'afdb',
    'millenniumchallenge',
    'worldbank',
    'iadb',
    'gac-amc',
    'unicef',
    'ec-echo',
    'ec-near',
    'gavi',
    'theglobalfund',
    'sida',
    'minbuza_nl',
    'usaid',
    'ec-devco',
    'be-dgd',
    'unitedstates',
    'bmz',
    'ausgov',
    'ebrd',
    'eib',
    'finland_mfa',
    'ifcwbg',
    'sdc_ch',
    'danida',
    'bmgf',
    'france',
    'afd',
    'aics',
    'norad',
    'irishaid',
    'jica',
    'odakorea',
    'maec',
    'fco',
    'unocha',
    'mfat',
    'mofa-jpn',

]

rows = []

for pub_name in pub_names:
    pub = registry.publishers.get(pub_name)
# print(pub)
    orgs = list(pub.organisations)
    for org in orgs:
        default_currency = org.etree.xpath('@default-currency')
        default_currency = default_currency[0] if default_currency != [] else 'No default_currency provided'
        name = org.etree.xpath('name/narrative/text()')
        name = name[0] if name != [] else 'No name provided'
        totals = org.etree.xpath('total-expenditure')
        for total in totals:
            period_start = total.xpath('period-start/@iso-date')[0]
            period_end = total.xpath('period-end/@iso-date')[0]
            value = total.xpath('value/text()')[0]
            currency = total.xpath('value/@currency')
            currency = currency[0] if currency != [] else default_currency
            rows.append({
                'name': name, 
                'period-start': period_start, 
                'period-end': period_end, 
                'value': value,
                'currency': currency,
            })
            
# print(rows)
            
with open('totalspend.csv', 'w') as handler:
    writer = csv.DictWriter(handler, fieldnames=fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)